# Orchestrate the nested chat

In [1]:
## import required libraries
import os
from autogen import AssistantAgent, ConversableAgent, UserProxyAgent
from dotenv import load_dotenv


load_dotenv() # load environment variables from .env file

False

In [2]:
## llm configuration
llm_config ={
                "model": "gpt-4o",
                "api_key": os.getenv("OPENAI_API_KEY")
            }

## Writer Agent

In [ ]:
writer = AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

## Task

In [ ]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


In [2]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

NameError: name 'writer' is not defined

In [ ]:
print(reply)

## Simple review of the post

In [ ]:
## reviewer agent
reviewer = AssistantAgent(
    name="Reviewer",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a reviewer. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [ ]:
res = reviewer.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

## Adding multi-level reviews using nested chats

In [1]:
SEO_reviewer = AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="As an SEO expert, your role is to analyze and enhance content for optimal search engine performance. "
        "Focus on providing actionable recommendations that boost rankings and drive organic traffic. "
        "Limit your feedback to 3 key points, ensuring they are specific and directly applicable. "
        "Start each review by introducing yourself as an SEO Reviewer.",
)

NameError: name 'autogen' is not defined

In [ ]:
grammatical_error_reviewer = AssistantAgent(
    name="Grammatical Error Reviewer",
    llm_config=llm_config,
    system_message="As a grammar specialist, your task is to meticulously examine content "
        "for grammatical errors, punctuation mistakes, and style inconsistencies. "
        "Provide up to 3 key points addressing the most significant grammatical issues. "
        "Ensure your feedback is specific, actionable, and includes examples where appropriate. "
        "Begin each review by introducing yourself as a Grammatical Error Reviewer.",
)

In [ ]:
ethics_reviewer = AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="As an ethics specialist, your role is to evaluate content for ethical integrity "
        "and identify any potential moral concerns. "
        "Provide up to 3 specific, actionable recommendations to address ethical issues. "
        "Ensure your feedback is concise and directly applicable. "
        "Start each review by introducing yourself as an Ethics Reviewer.",
)

In [ ]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrating the nested chat

In [ ]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": grammatical_error_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [ ]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [ ]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

## Get summary

In [ ]:
print(res.summary)